<a href="https://colab.research.google.com/github/oscarserc/TESIS/blob/main/PASO_3e___TRANSFORMERS_En_construccion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#UTILIZACIÓN DE TÉCNICAS DE APRENDIZAJE AUTOMÁTICO PARA LA DETECCIÓN DE LOS PUNTOS DE CONFLICTIVIDAD EN LA RED SOCIAL.

La base de datos que emplearemos será de unas 70.000 observaciones. En concreto:

· 72.000 observaciones

· Para ello generamos una 1800 **simulaciones**, y guardamos unos 40 **frames** de cada una de ellas

· Además como **target** guardamos la matriz con la IDENTIDAD de todos y cada uno de los AGENTES.



## **Prueba con los Mecanismos de atención del TRANSFORMER**

## Utilización de la librería PyTorch

Vamos a construir primero una matriz que simule un embeding. Algunos de los elementos de la secuencia serán más similares que otros entre sí.

In [ ]:

import torch

a = torch.arange(8).reshape(2,4)
a


tensor([[0, 1, 2, 3],
        [4, 5, 6, 7]])

In [ ]:

b = a.view(2,2,2)


In [ ]:

b


tensor([[[0, 1],
         [2, 3]],

        [[4, 5],
         [6, 7]]])

In [ ]:

b[0,:,:]


tensor([[0, 1],
        [2, 3]])

In [ ]:

import torch

a = torch.zeros(3,5)
a[0,:] = 1
a[1,:] = 2
a[2,:] =3
a


tensor([[1., 1., 1., 1., 1.],
        [2., 2., 2., 2., 2.],
        [3., 3., 3., 3., 3.]])

In [ ]:

a.reshape(5,3)


tensor([[1., 1., 1.],
        [1., 1., 2.],
        [2., 2., 2.],
        [2., 3., 3.],
        [3., 3., 3.]])

In [ ]:

a.transpose(0,1)


tensor([[1., 2., 3.],
        [1., 2., 3.],
        [1., 2., 3.],
        [1., 2., 3.],
        [1., 2., 3.]])

In [ ]:

b = a.unsqueeze(0)
b.shape


torch.Size([1, 3, 5])

In [ ]:

b


tensor([[[1., 1., 1., 1., 1.],
         [2., 2., 2., 2., 2.],
         [3., 3., 3., 3., 3.]]])

In [ ]:

c = b.permute(2,0,1)
c


tensor([[[1., 2., 3.]],

        [[1., 2., 3.]],

        [[1., 2., 3.]],

        [[1., 2., 3.]],

        [[1., 2., 3.]]])

In [ ]:

c.shape


torch.Size([5, 1, 3])

## **Mecanismos de atención**

Vamos a construir primero una matriz que simule un embeding. Algunos de los elementos de la secuencia serán más similares que otros entre sí.

In [ ]:

import torch

# Supongamos:
# · Longitud de la secuencia:  T = 5
# · Dimensión del embedding :  D = 10
T = 5
D = 12
x = torch.zeros(5,12)
x[0,:] = 1
x[1,:] = 2
x[2,:] = 3
x[3,:] = 2
x[4,:] = 6
x


tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.]])

In [ ]:

n_embd  = D
n_heads = 2

key   = torch.nn.Linear(n_embd, n_embd//n_heads)
query = torch.nn.Linear(n_embd, n_embd//n_heads)
value = torch.nn.Linear(n_embd, n_embd//n_heads)
key(x)


tensor([[-0.5899, -0.4960, -0.9899, -0.2596, -0.6884, -0.2277],
        [-1.1424, -0.9016, -1.7010, -0.7512, -1.1408, -0.6714],
        [-1.6949, -1.3073, -2.4122, -1.2428, -1.5931, -1.1152],
        [-1.1424, -0.9016, -1.7010, -0.7512, -1.1408, -0.6714],
        [-3.3524, -2.5243, -4.5456, -2.7176, -2.9502, -2.4464]],
       grad_fn=<AddmmBackward0>)

In [ ]:

x_batch = torch.unsqueeze(x, dim=0)
x_batch.shape

dim_qkv = D // n_heads

#B, T, dim_Embd = x.size()
x.view(T, dim_qkv, n_heads)


tensor([[[1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.]],

        [[2., 2.],
         [2., 2.],
         [2., 2.],
         [2., 2.],
         [2., 2.],
         [2., 2.]],

        [[3., 3.],
         [3., 3.],
         [3., 3.],
         [3., 3.],
         [3., 3.],
         [3., 3.]],

        [[2., 2.],
         [2., 2.],
         [2., 2.],
         [2., 2.],
         [2., 2.],
         [2., 2.]],

        [[6., 6.],
         [6., 6.],
         [6., 6.],
         [6., 6.],
         [6., 6.],
         [6., 6.]]])

In [ ]:

# CONTENIDO de cada una de las CABEZAS
x.view(T, dim_qkv, n_heads).transpose(0,2)


tensor([[[1., 2., 3., 2., 6.],
         [1., 2., 3., 2., 6.],
         [1., 2., 3., 2., 6.],
         [1., 2., 3., 2., 6.],
         [1., 2., 3., 2., 6.],
         [1., 2., 3., 2., 6.]],

        [[1., 2., 3., 2., 6.],
         [1., 2., 3., 2., 6.],
         [1., 2., 3., 2., 6.],
         [1., 2., 3., 2., 6.],
         [1., 2., 3., 2., 6.],
         [1., 2., 3., 2., 6.]]])

#**DETALLES**

In [ ]:

import torch

# Supongamos:
# · Longitud de la secuencia:  T = 5 (5 tokens)
# · Dimensión del embedding :  D = 12

T = 5
D = 12

x = torch.zeros(5,D)
x[0,:] = 1
x[1,:] = 2
x[2,:] = 3
x[3,:] = 2
x[4,:] = 6
x

# SUPONEMOS QUE ES LA SALIDA YA DE LA RED NEURONAL


tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.]])

In [ ]:

# Supongamos que tenemos 4 cabezas.
# El embedding (D=12) lo representamos como una matriz
# Usaremos h = 4 cabezas

# Entoces tenemos que cada TOKEN (T=5) quedará representado
# como una matriz. El EMBEDDING se ha fragmentado para dividir
# entre varias unidades de procesamiento que puedan centrarse
# en aspectos y matices particulares del INPUT.

# D = 12  y tomamos h = 4 cabezas
# luego cada una va a trabajar con una pequeña fracción del
# espacio. en concreto: d_h = 3.
h = 4
x.view(5,3,h)


tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[2., 2., 2., 2.],
         [2., 2., 2., 2.],
         [2., 2., 2., 2.]],

        [[3., 3., 3., 3.],
         [3., 3., 3., 3.],
         [3., 3., 3., 3.]],

        [[2., 2., 2., 2.],
         [2., 2., 2., 2.],
         [2., 2., 2., 2.]],

        [[6., 6., 6., 6.],
         [6., 6., 6., 6.],
         [6., 6., 6., 6.]]])

In [ ]:

x.view(5,3,h).transpose(0,2)


tensor([[[1., 2., 3., 2., 6.],
         [1., 2., 3., 2., 6.],
         [1., 2., 3., 2., 6.]],

        [[1., 2., 3., 2., 6.],
         [1., 2., 3., 2., 6.],
         [1., 2., 3., 2., 6.]],

        [[1., 2., 3., 2., 6.],
         [1., 2., 3., 2., 6.],
         [1., 2., 3., 2., 6.]],

        [[1., 2., 3., 2., 6.],
         [1., 2., 3., 2., 6.],
         [1., 2., 3., 2., 6.]]])

In [ ]:

# CABEZA h = 1     1/4 (indice = 0)
# ============

# Embedding ------> d = 12/4 = 3 (La parte que le
#                                 haya tocado de
#                                 los D=12)
# Tokens ---------> T = 5
x.view(5,3,h).transpose(0,2)[0,:,:]


tensor([[1., 2., 3., 2., 6.],
        [1., 2., 3., 2., 6.],
        [1., 2., 3., 2., 6.]])

In [ ]:

y = x.view(5,3,h).transpose(0,2)
y

tensor([[[1., 2., 3., 2., 6.],
         [1., 2., 3., 2., 6.],
         [1., 2., 3., 2., 6.]],

        [[1., 2., 3., 2., 6.],
         [1., 2., 3., 2., 6.],
         [1., 2., 3., 2., 6.]],

        [[1., 2., 3., 2., 6.],
         [1., 2., 3., 2., 6.],
         [1., 2., 3., 2., 6.]],

        [[1., 2., 3., 2., 6.],
         [1., 2., 3., 2., 6.],
         [1., 2., 3., 2., 6.]]])

In [ ]:

y.transpose(0,2)


tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[2., 2., 2., 2.],
         [2., 2., 2., 2.],
         [2., 2., 2., 2.]],

        [[3., 3., 3., 3.],
         [3., 3., 3., 3.],
         [3., 3., 3., 3.]],

        [[2., 2., 2., 2.],
         [2., 2., 2., 2.],
         [2., 2., 2., 2.]],

        [[6., 6., 6., 6.],
         [6., 6., 6., 6.],
         [6., 6., 6., 6.]]])

In [ ]:

# ver si hay ERROR
# ----------------

# En el de SENSIO aparece (1,2), no el (1,3) que era
# el que había cambiado previamente
y.transpose(0,2).contiguous().view(5,3*h)


tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.]])